In [79]:
"to run this code, you need to have standalone EMBOSS (getorf), BLAST, and HMMER installed"

'to run this code, you need to have standalone EMBOSS (getorf), BLAST, and HMMER installed'

In [108]:
import subprocess
import os
from glob import glob
from Bio import SearchIO
from Bio import SeqIO

### Extract ORFs from genomes

In [81]:
def run_getorf(genome, orf_file):
    command = ["getorf", "-sequence", genome, "-outseq", orf_file, "-table", "0", "-minsize", "360", "-find", "1"] #adjust (120 aa, START to STOP, standard gen. code)
    subprocess.run(command, check=True)

outdir = "/home/majnusova/all/projects/plv/data/eustig_orfs/" #adjust
genomes = glob("/home/majnusova/all/projects/plv/data/genomes_test/*.fasta") #adjust - this is just a test dataset

for genome in genomes:
    orf_file = f"{outdir}{genome.split('/')[-1].replace('.fasta', '_orfs.fa')}"
    run_getorf(genome, orf_file)

Find and extract open reading frames (ORFs)
Find and extract open reading frames (ORFs)
Find and extract open reading frames (ORFs)
Find and extract open reading frames (ORFs)


### hmmsearch with HMM against those extracted ORFs

In [96]:
# running hhmmsearch against multiple databases at once and saving the resulting files using the .hmmout suffix
def run_hmmsearch(orf_file, hmm_model, output_file):
    command = ["hmmsearch", "-o", output_file, hmm_model, orf_file]
    subprocess.run(command, check=True)

hmm_model = "/home/majnusova/all/projects/bilabrum/data/hmm/Packiging_ATPase.hmm" #adjust
output_directory = "/home/majnusova/all/projects/plv/data/eustigs_hmmout/" #adjust
orf_files = glob("/home/majnusova/all/projects/plv/data/eustig_orfs/*_orfs.fa") #adjust: list of orf databases


# iterate over the list of database files and run hmmsearch 
for orf_file in orf_files:
    output_file = f"{output_directory}{orf_file.split('/')[-1].replace('_orfs.fa', '_atp.hmmout')}" 
    run_hmmsearch(orf_file, hmm_model, output_file) #needs to be nested inside the loop!


### saving IDs of homologs above inclusion threshold found by HMMER (in all the genomes)

In [97]:
# saving IDs of sequences above the inclustion treshold into ids_list
ids_list_orfs = []
for file in glob("/home/majnusova/all/projects/plv/data/eustigs_hmmout/*_atp.hmmout"):
    hmmer_file = SearchIO.read(file, "hmmer3-text")
    for record in hmmer_file:
        if record.is_included:
            ids_list_orfs.append(record.id)
len(ids_list_orfs) 


348

In [98]:
print(ids_list_orfs)

['NODE_5080_length_5696_cov_26.6876_1', 'NODE_381_length_27838_cov_14.2215_25', 'NODE_7293_length_4272_cov_22.9291_3', 'NODE_5397_length_5445_cov_37.4879_2', 'NODE_6066_length_4954_cov_14.3046_4', 'NODE_847_length_17049_cov_12.9829_3', 'NODE_7343_length_4249_cov_22.2454_2', 'NODE_19673_length_1550_cov_34.6107_1', 'NODE_1409_length_12728_cov_15.5927_2', 'NODE_1574_length_11923_cov_14.5027_1', 'NODE_711_length_18631_cov_14.7806_4', 'NODE_15445_length_2031_cov_23.9084_2', 'NODE_15112_length_2083_cov_16.6144_1', 'NODE_12470_length_2567_cov_17.4088_2', 'NODE_34064_length_837_cov_18.9655_1', 'NODE_4057_length_6629_cov_27.157_1', 'NODE_865_length_16875_cov_12.6059_4', 'NODE_869_length_16855_cov_12.4798_4', 'NODE_6477_length_4696_cov_21.0899_1', 'NODE_17368_length_1781_cov_15.3621_1', 'NODE_18418_length_1671_cov_21.013_1', 'NODE_352_length_29307_cov_12.7034_4', 'NODE_31105_length_924_cov_16.8895_1', 'NODE_4114_length_6570_cov_18.8181_3', 'NODE_64132_length_412_cov_37.521_1', 'NODE_21363_length

### Extracting scaffolds with the viral homologs found by HMMER (ids_list), scaffolds shorter than 18000 nt are discarded

In [99]:
# ids_list contains IDs of ORFs, not scaffolds! this list needs to be modified accordingly
ids_list_scaffolds = []
for id in ids_list_orfs:
    id_modified = id.rsplit('_', 1)[0] # removing the number of orf -> id of the scaffold
    ids_list_scaffolds.append(id_modified)
print(ids_list_scaffolds)

['NODE_5080_length_5696_cov_26.6876', 'NODE_381_length_27838_cov_14.2215', 'NODE_7293_length_4272_cov_22.9291', 'NODE_5397_length_5445_cov_37.4879', 'NODE_6066_length_4954_cov_14.3046', 'NODE_847_length_17049_cov_12.9829', 'NODE_7343_length_4249_cov_22.2454', 'NODE_19673_length_1550_cov_34.6107', 'NODE_1409_length_12728_cov_15.5927', 'NODE_1574_length_11923_cov_14.5027', 'NODE_711_length_18631_cov_14.7806', 'NODE_15445_length_2031_cov_23.9084', 'NODE_15112_length_2083_cov_16.6144', 'NODE_12470_length_2567_cov_17.4088', 'NODE_34064_length_837_cov_18.9655', 'NODE_4057_length_6629_cov_27.157', 'NODE_865_length_16875_cov_12.6059', 'NODE_869_length_16855_cov_12.4798', 'NODE_6477_length_4696_cov_21.0899', 'NODE_17368_length_1781_cov_15.3621', 'NODE_18418_length_1671_cov_21.013', 'NODE_352_length_29307_cov_12.7034', 'NODE_31105_length_924_cov_16.8895', 'NODE_4114_length_6570_cov_18.8181', 'NODE_64132_length_412_cov_37.521', 'NODE_21363_length_1410_cov_17.5911', 'NODE_729_length_18418_cov_17.5

#### to be able to extract the scaffolds, blastable databases need to be created first (for genome and orfs)

In [100]:
def makeblastdb(file_path, db_type, blastables_outdir):
    db_name = f"{blastables_outdir}/{file_path.split('/')[-1].split('.')[0]}"
    command = ["makeblastdb", "-in", file_path, "-dbtype", db_type, "-out", db_name, "-parse_seqids"]
    subprocess.run(command, check=True)

blastables_outdir = "/home/majnusova/all/projects/plv/data/blastables/"


for genome_path in genomes:
    makeblastdb(genome_path, "nucl", blastables_outdir) # misto file_path volame s genome_path

for orf_file_path in orf_files:
    makeblastdb(orf_file_path, "prot", blastables_outdir)




Building a new DB, current time: 12/27/2023 19:20:40
New DB name:   /home/majnusova/all/projects/plv/data/blastables/Chic10_scaffolds
New DB title:  /home/majnusova/all/projects/plv/data/genomes_test/Chic10_scaffolds.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/majnusova/all/projects/plv/data/blastables/Chic10_scaffolds
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 758531 sequences in 9.90868 seconds.




Building a new DB, current time: 12/27/2023 19:20:52
New DB name:   /home/majnusova/all/projects/plv/data/blastables/Vischeria_Fay-Wei_consensus
New DB title:  /home/majnusova/all/projects/plv/data/genomes_test/Vischeria_Fay-Wei_consensus.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/majnusova/all/projects/plv/data/blastables/Vischeria_Fay-Wei_consensus
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 171 sequences in 0.399329 seconds.



# VYRESIT PROC GETORF GENERUJE ORFY Z FAY WEIOVY VISCHERIE SE SPATNYMI IDS - PAK JE NEBERE ANI BLASTDBCM

In [107]:
import os
import subprocess
def run_blastdbcmd(id, genome_db, outscaffold):
    command = ["blastdbcmd", "-entry", str(id), "-db", genome_db, "-out", outscaffold]
    result = subprocess.run(command, text=True, capture_output=True)
    if result.returncode != 0:
        print(f"Warning: ID {id} not found in database {genome_db}.")
    # Check if the file was created and is empty
    elif os.path.exists(outscaffold) and os.path.getsize(outscaffold) == 0:
        print(f"Notice: No data for ID {id}, removing empty file.")
        os.remove(outscaffold)

def run_blastdbcmd(id, genome_db, outscaffold):
    command = ["blastdbcmd", "-entry", str(id), "-db", genome_db, "-out", outscaffold]
    result = subprocess.run(command, text=True, capture_output=True)
    if result.returncode != 0:
        print(f"Warning: ID {id} not found in database {genome_db}.")
        return False
    return True

scaffolds_dir = "/home/majnusova/all/projects/plv/data/eve_outscaffolds/"
blastables_outdir = "/home/majnusova/all/projects/plv/data/blastables/"
x = "_orfs."

for id in ids_list_scaffolds:
    for filename in os.listdir(blastables_outdir):
        if x not in filename and filename.endswith('.nsq'):  # Look for .nsq files which are part of BLAST db
            base_name = filename.split('.')[0]  # Assume the base name of the database is the filename without extension
            genome_db = os.path.join(blastables_outdir, base_name)  # Construct the path to the database
            outscaffold = f"{scaffolds_dir}/{id}_EVE.fasta"
            success = run_blastdbcmd(id, genome_db, outscaffold)
            if success:
                break  # If we successfully extracted the scaffold, no need to check other databases


In [113]:
def run_blastdbcmd(id, genome_db, outscaffold):
    command = ["blastdbcmd", "-entry", str(id), "-db", genome_db, "-out", outscaffold]
    result = subprocess.run(command, text=True, capture_output=True)

    if result.returncode == 0:
        if os.path.getsize(outscaffold) > 0:
            record = SeqIO.read(outscaffold, "fasta")
            if len(record.seq) < 18000:
                print(f"Notice: Scaffold {id} is shorter than 15000 nucleotides, removing file.")
                os.remove(outscaffold)
                return False
            return True
        else:
            print(f"Notice: The file created for ID {id} is empty, removing.")
            os.remove(outscaffold)
            return False
    else:
        print(f"Warning: ID {id} not found in database {genome_db}.")
        return False

scaffolds_dir = "/home/majnusova/all/projects/plv/data/eve_outscaffolds/"
blastables_outdir = "/home/majnusova/all/projects/plv/data/blastables/"
x = "_orfs."


for id in ids_list_scaffolds:
    for filename in os.listdir(blastables_outdir):
        if x not in filename and filename.endswith('.nsq'):
            base_name = filename.split('.')[0]
            genome_db = os.path.join(blastables_outdir, base_name)
            outscaffold = f"{scaffolds_dir}/{id}_EVE.fasta"
            if run_blastdbcmd(id, genome_db, outscaffold):
                break


Notice: Scaffold NODE_5080_length_5696_cov_26.6876 is shorter than 15000 nucleotides, removing file.
Notice: Scaffold NODE_7293_length_4272_cov_22.9291 is shorter than 15000 nucleotides, removing file.
Notice: Scaffold NODE_5397_length_5445_cov_37.4879 is shorter than 15000 nucleotides, removing file.
Notice: Scaffold NODE_6066_length_4954_cov_14.3046 is shorter than 15000 nucleotides, removing file.
Notice: Scaffold NODE_847_length_17049_cov_12.9829 is shorter than 15000 nucleotides, removing file.
Notice: Scaffold NODE_7343_length_4249_cov_22.2454 is shorter than 15000 nucleotides, removing file.
Notice: Scaffold NODE_19673_length_1550_cov_34.6107 is shorter than 15000 nucleotides, removing file.
Notice: Scaffold NODE_1409_length_12728_cov_15.5927 is shorter than 15000 nucleotides, removing file.
Notice: Scaffold NODE_1574_length_11923_cov_14.5027 is shorter than 15000 nucleotides, removing file.
Notice: Scaffold NODE_15445_length_2031_cov_23.9084 is shorter than 15000 nucleotides, r